# Exercise 3: Multiple Requests and Callbacks

In this exercise, you will implement inference with multiple requests using callbacks. 

The workload will be once again vehicle detection, but on a video this time. 
Specifically, your application will count the cars in the frame and report three metrics: maximum number of cars in one frame, minimum number of cars in one frame, and average number of cars in all frames.
Run the fllowing cell to see the video.

In [ ]:
from IPython.core.display import HTML
!ln -sf /data/reference-sample-data/object-detection-python/cars_1900.mp4 
HTML("<video alt=\"\" controls autoplay height=\"480\"><source src=\"cars_1900.mp4\" type=\"video/mp4\" /></video>")

### Important! The quiz will ask you how the average number of vehicles detected in the last step.


## Implementation

The video course covered some potential implementations using the `wait()` function, including the zero timeout wait.
While the zero timeout example in the video works well, it goes through all the requests over and over again until one of them is done.
In this exercise, you will implement multiple inference that simply waits for the first finished slot using Python queues and inference callbacks

Python queues have couple of interesting features that make them work well with the multiple request inference workload.
One is that Python queues are thread-safe. 
WIthout going in to too much detail, this means that the queue is safe to use in an asynchronous setting, like our requests.
The second feature is the `get()` function (the "pop" function).
If the queue is empty when `get()` is called, it will wait until an item becomes available.

The trick is to have a queue of *available request slots* that the main loop waits for, so that the main loop can wait on the queue instead of checking for the status of every slot.
When individual requests are done, they will need to add their own slot id number to the queue.
This is where we take advantage of the callback function, which is called right when an inference is completed.

## (Optional) Step 0: Python queue

This section is designed to give you a brief introduction to Python queue. 
If you are already familiar, skip to step 1.

Python queue are data structures that are accessed in First In First Out (FIFO) order.
When used in an asynchrnous workload, this can be used to access the jobs as they complete.

The following is a brief example of using queue in an asynchronous setting.
This example uses threading instead of inference engine so to keep the example simple.
Each thread sleeps for some time, and then puts a tuple containing the id of the thread and how long it slept for.
The main thread will wait on the queue, and print out the contents of the tuple.

In [ ]:
import queue
import threading
import time

# Sample asynchronous workload that simply sleeps, and then places ID in queue
def foo(q, myid, timeout):
    time.sleep(timeout)
    q.put((myid, timeout))

# Creating the queue for completed tasks
completion_queue = queue.Queue()

# Create and start two tasks
t1 = threading.Thread(target=foo, args=(completion_queue, 1, 3))
t2 = threading.Thread(target=foo, args=(completion_queue, 2, 1))
t1.start()
t2.start()

# Print tasks as they complete
completed_id, timeout = completion_queue.get()
print("task {} completed after sleeping for {} second(s)".format(completed_id, timeout))
completed_id, timeout = completion_queue.get()
print("task {} completed after sleeping for {} second(s)".format(completed_id, timeout))


# Confirming the threads are completed. Not necessary, but good practice.
t1.join()
t2.join()

Notice that the task 2 had a shorter timeout and completed first, and it was printed immediately without waiting for task 1 to complete.
Additionally, notice that I did not have to specify any id in the `get()` function.
We will adapt this for inference engine


## Step 1: Downloading model

We will once again use the `vehicle-detection-adas-0002` model.
Using the model downloader, download the FP32 and FP16 versions of the model.

In [ ]:
%%bash
/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --help

## Step 2: Helper functions

Begin by writing various helper functions for use in the main loop.
Complete the `utils.py` file by following the instructions.

*(2.1)* Complete the `prepImage()` function, which is used to prepare the image for inference. The code here should be the exact same as in exercise 1.

*(2.2)* In `createExecNetwork()` function, create an instance of IECore object and load the CPU plugin specified by the variable `extension`. The solution is identical to the implementationin exercise 1. Optionally, add a if check to see if "CPU" appears in the device list. While it is safe toload the extension even if you are not using CPU, it is a good practice to add a if check to not load unnecessary extensions.

*(2.3)* In `createExecNetwork()` function, create an instance of ExecutableNetwork from `ie_net` with the optimal number of requests and return it. The method for this is demonstrated in the slides for video 2 of course 2.

*(2.4)* In `setCallbackAndQueue()` function, add a callback function called `callbackFunc` to each of the request slots. We will be defining this function in step (2.5). To do this, first create a dictinary that contains the queue and the request slot id. The key to use for this dictionary is up to you. Then call the  `set_completion_callback()` method for the requests to add the `callbackFun` (note the lack of parethesis). For more details, see the slides for course 2 video 6.

*(2.5)* In `callbackFunc()` function, add a tuple containing the request slot ID and the status code for the inference. See the queue usage in `setCallbackAndQueue()` for how to add this tuple. Remember that py_data is the dictionary you passed in in the previoud step.

In [ ]:
%%writefile utils.py
import cv2
from openvino.inference_engine import IECore, IENetwork

# Prepares image for inference by reshaping and transposing.
# inputs:
#     orig_image - numpy array containing the original, unprocessed image
#     ie_net     - IENetwork object 
def prepImage(orig_image, ie_net):
    
    ##! (2.1) Find n, c, h, w from ie_net !##
    
    input_image = cv2.resize(orig_image, (w, h))
    input_image = input_image.transpose((2, 0, 1))
    input_image.reshape((n, c, h, w))

    return input_image

# Processes the result. Returns the number of detected vehices in the image.
# inputs:
#    detected_obects - numpy array containing the ooutput of the model
#    prob_threashold - Required probability for "detection"
# output:
#    Number of vehices detected.
def getCount(detected_objects, prob_threshold=0.5):
    detected_count = 0
    for obj in detected_objects[0][0]:
        # Draw only objects when probability more than specified threshold
        if obj[2] > prob_threshold:
            detected_count+=1
    return detected_count


# Create ExecutableNetwork with the optimial number of requests for a given device.
# inputs:
#    ie_net - IENetwork object to use
#    device - String to use for device_name argument.
# output:
#    ExecutabeNetwork object
def createExecNetwork(ie_net, device):
    ##! (2.2) Create IECore !##
    
    ##! (2.2) Load the CPU plugin (optional: check if it is needed)!##
    extension = '/opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_avx2.so'

    ##! (2.3) Create ExecutableNetwork object and find the optimizal number of requests !##

    ##! (2.3) Recreate IECore and with num_requests set to optimial number of requests !##
    
    ##! (2.3) return the ExecutableNetwork !##

    
# Set callback functions for the inference requests.
# inputs:
#    exec_net - ExecutableNetwork object to modify
#    c_queue  - Python queue to put the slot ID in
def setCallbackAndQueue(exec_net, c_queue):
    for req_slot in range(len(exec_net.requests)):
        ##! (2.4) Create a dictionary for py_data to pass in the queue and ID !###

        ##! (2.4) Set the completion callback with the arguments for each reqeust !##
        
        # Initializing the queue. The second item of the tuple is the status of the previous 
        #  inference. But as there is no previous inference right now, setting the status to None.
        c_queue.put((req_slot, None))
    
# Callback function called on completion of the inference.
# inputs:
#    status  - status code for the inference.
#    py_data - dictionary arguments passed into the function
def callbackFunc(status, py_data):
    try:
        ##! (2.5) Add a tuple (id, status) to queue here !##
    except:
        print("There was an issue in callback")

## Step 3: Main loop

Now write the main loop. 
Complete the `main.py` file by following the instructions.

*Note* Many of the variables are already placed and set to None. This is because these variables are used in other parts of the code that have been provided to you. So do not change th name of the variable, but instead replace None with code specified by the instructions.


*(3.1)* Create the IENetwork object with FP16 version the `vehicle-detection-adas-0002` model that we have downloaded earlier. Then find the name of the input layer and output layer.

*(3.2)* Run the `getCount()` function to get the number of vehicles in the inference result from the slot that completed (`req_slot`). The `getCount()` function takes a numpy array containing the result of the `vehicle-detection-adas-0002` model. Remember that the `output` attribute of the InferRequest object is a dictionary. Refer to exercise 1 for how to get the output of the inference.

*(3.3)* Start the next asynchrounus inference on the completed slot (`req_slot`) with `start_async()` method of the ExecutableNetwork. See exercise 1 for how to use `start_async()`.

*(3.4)* Repeat step 3.2 again for the edge case handlnig. 

In [ ]:
%%writefile main.py
import cv2
import sys
import queue
from openvino.inference_engine import IECore, IENetwork
from utils import *

device = sys.argv[1]

##! (3.1) Create IENetwork object from vehicle-detection-adas-0002 !##
ie_net = None

##! (3.1) Get the name of input and output layers. There is only one of each. !##
input_layer  = None
output_layer = None

# Create ExecutableNetwork object using createExecNetwork in utils.py 
exec_net = createExecNetwork(ie_net, device)
print("ExecutableNetwork created with {} requests.".format(len(exec_net.requests)))

# Set the callback functions using setCallbackAndQueue() in utils.py 
c_queue = queue.Queue()
setCallbackAndQueue(exec_net, c_queue)

# Stats for processing
max_vehicles = 0
min_vehicles = 999      # this is safe as the max number of detectable objects is 200
sum_vehicles = 0
num_frames = 0
# Loading the data from a video
input_video = "/data/reference-sample-data/object-detection-python/cars_1900.mp4"
cap = cv2.VideoCapture(input_video)
while cap.isOpened():
    # Read the next frame
    ret, next_frame = cap.read()
    # Condition for the end of video
    if not ret:
        break
        
    ##! preprocess next_frame using prepImage from utils.py !##
    input_frame = prepImage(next_frame, ie_net) 
    
    # using get to wait for the nextslot ID. Here we are setting a timeout of 30 seconds in case 
    #  there are issues with the callback and queue never gets populated. With timeout, this function
    #  will error out  with "Empty"
    req_slot, status = c_queue.get(timeout=30)
    
    if status == 0:
        ##! (3.2) Postprocess result from the request slot using getCount function from utils.py !##
        num_vehicles = None
        
        max_vehicles = max(num_vehicles, max_vehicles)
        min_vehicles = min(num_vehicles, min_vehicles)
        sum_vehicles += num_vehicles
        num_frames += 1
        
    # Recall that None is what we set for the first time initializeation of queue, so we catch everything else.
    elif not status is None:
        print("There was error in processing an image")

    ##! (3.3) Start the next inference on the now open slot. !##
    

# Handle the remaining images.
#  first we wait for all request slots to complete
for req in exec_net.requests:
    req.wait()

# Handle remaining results 
while not c_queue.empty():
    req_slot, status = c_queue.get(timeout=30)
    
    if status == 0:
        ##! (3.4) Postprocess result from the request slot using getCount function from utils.py !##
        num_vehicles = None
        
        max_vehicles = max(num_vehicles, max_vehicles)
        min_vehicles = min(num_vehicles, min_vehicles)
        sum_vehicles += num_vehicles
        
    # Recall that None is what we set for the first time initializeation of queue, so we catch everything else.
    elif not status is None:
        print("There was error in processing an image")
        
# Finally, reporting results.
print("Maximum number of cars detected: {}".format(max_vehicles))
print("Minimum number of cars detected: {}".format(min_vehicles))
print("average number of cars detected: {:.3g}".format(sum_vehicles/num_frames))

## Run the job

Finally, let us try to run the workload. 
The following code will submit a job to the VPU system. 
The commands, as well as the utility `waitForJob()` function arethe same as the exercise 1.

**Note:** The toolkit is very verbose when using MYRIAD systems, so you may get a lot of additional output beyond what you are expecting. 

**The final average vehicles detected to the third decimal will be asked in the quiz.**

In [ ]:
from notebook_utils import waitForJob
job_name = !echo "python3 main.py HDDL" | qsub -d `pwd` -N objdet -l nodes=1:iei-mustang-v100-mx8
waitForJob(job_name)

Congratulations! You have just ran multiple requests in parallel. 
From the output, you can see that up to 32 requests could be ran in parallel. 
Though in practice a fewer number is actually used because the CPU can not keepup with the preprocessing.

**The final average vehicles detected to the third decimal will be asked in the quiz.**